In [49]:
import numpy as np
import math
import latexify
import blob_utils

In [50]:
@latexify.function(use_math_symbols=True)
def LoG(x, y, sigma):
    return ((- 1 / (np.pi * sigma**4) ) * (1 - (x**2 + y**2) / (2 * sigma**2) ) ) * np.exp(- (x**2 + y**2) / (2 * sigma**2))
LoG

In [51]:
KERNEL_SIZE = 51
SIGMA =14
PATH = "easy.png"
kernel = blob_utils.get_kernel(fun=LoG, size=KERNEL_SIZE, sigma=SIGMA)

In [52]:
import cv2
rgb_image = cv2.imread(PATH)
gray_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2GRAY) 
cv2.imwrite("grayscale_"+PATH, gray_image)

True

In [53]:
print(gray_image.shape)
#padded_image = blob_utils.pad_image(image=image, blob_radius=KERNEL_SIZE//2)
padded_image = gray_image
print(padded_image.shape)

(1040, 1388)
(1040, 1388)


In [54]:
convolved_image = blob_utils.convolve_image(padded_image=padded_image, kernel=kernel)

cv2.imwrite(
    "convolved_"+PATH,
    np.interp(
        convolved_image, 
        (convolved_image.min(), convolved_image.max()),
        (0, 255)
    )
)

True

In [55]:
blob_radius = int (np.sqrt(2) * SIGMA) + 1

centers = []

p = np.percentile(convolved_image, [5, 95])
lower_treshold = p[0]
upper_treshold = p[1]

for i in range(convolved_image.shape[0]):
    for j in range(convolved_image.shape[1]):
        
        local_minimum, local_maximum = blob_utils.strictly_local_min_max(convolved_image, center=(i, j), radius=blob_radius)
        
        if (convolved_image[i,j] < local_minimum) and (convolved_image[i, j] < lower_treshold):
            centers.append( ( i + KERNEL_SIZE//2, j + KERNEL_SIZE//2, convolved_image[i][j], "min"))
        if (convolved_image[i,j] > local_maximum) and (convolved_image[i, j] > upper_treshold):
            centers.append( ( i+ KERNEL_SIZE//2, j+ KERNEL_SIZE//2, convolved_image[i][j], "max"))

print(len(centers))


406


In [56]:
for (i, j , _, tipo) in centers:
    if tipo=="min":
        cv2.circle(rgb_image, (j, i), blob_radius, (0, 0, 255), thickness=1, lineType=2)
    else:
        cv2.circle(rgb_image, (j, i), blob_radius, (0, 255, 0), thickness=1, lineType=2)

cv2.imwrite("blobs_"+PATH, rgb_image)

True